# Laboratorio 8
### Oscar Fernando López Barrios
### Carné 20679

In [15]:
!pip install adversarial-robustness-toolbox

In [16]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/SDS/malimg_paper_dataset_imgs"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print(f"Using GPU: {gpus[0].name}")
    except RuntimeError as e:
        print(e)

Physical devices cannot be modified after being initialized


In [18]:
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(
    path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

Found 9339 images belonging to 25 classes.


Para la realización de este laboratorio se utilizó el modelo que se obtuvo mediante el laboratorio 7.

### Ataque y Defensa de Fast Gradient Method

In [19]:
import numpy as np
from tensorflow.keras.models import load_model
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tf.compat.v1.disable_eager_execution()

model = load_model('modelo_lab_8.h5')

classifier = KerasClassifier(model=model, clip_values=(0, 1))

attack = FastGradientMethod(estimator=classifier, eps=0.1)

def generate_and_mix_adversarial_batch(generator, attack, batch_size=32):
    x_batch, y_batch = next(generator)
    x_adv = attack.generate(x=x_batch)

    x_train_mixed = np.concatenate((x_batch, x_adv), axis=0)
    y_train_mixed = np.concatenate((y_batch, y_batch), axis=0)

    indices = np.arange(x_train_mixed.shape[0])
    np.random.shuffle(indices)
    x_train_mixed = x_train_mixed[indices]
    y_train_mixed = y_train_mixed[indices]

    return x_train_mixed, y_train_mixed

x_train_mixed, y_train_mixed = generate_and_mix_adversarial_batch(train_generator, attack)

model.fit(x_train_mixed, y_train_mixed, epochs=10, batch_size=32)

model.save('modelo_lab_8_reentrenado.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Train on 64 samples
Epoch 1/10
64/64 [==============================] - 0s 2ms/sample - loss: 6.2356 - accuracy: 0.5156
Epoch 2/10
64/64 [==============================] - 0s 310us/sample - loss: 4.9314 - accuracy: 0.3906
Epoch 3/10
64/64 [==============================] - 0s 294us/sample - loss: 2.0848 - accuracy: 0.4375
Epoch 4/10
64/64 [==============================] - 0s 322us/sample - loss: 1.4874 - accuracy: 0.5625
Epoch 5/10
64/64 [==============================] - 0s 300us/sample - loss: 1.3487 - accuracy: 0.5625
Epoch 6/10
64/64 [==============================] - 0s 299us/sample - loss: 1.1818 - accuracy: 0.7188
Epoch 7/10
64/64 [==============================] - 0s 305us/sample - loss: 1.0726 - accuracy: 0.6562
Epoch 8/10
64/64 [==============================] - 0s 343us/sample - loss: 1.0817 - accuracy: 0.6719
Epoch 9/10
64/64 [==============================] - 0s 293us/sample - loss: 1.0464 - accuracy: 0.6562
Epoch 10/10
64/64 [==============================] - 0s 314us/sa

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
datagen_test = ImageDataGenerator(rescale=1./255)
test_generator = datagen_test.flow_from_directory(
    path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 9339 images belonging to 25 classes.


In [21]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.models import load_model

model = load_model('modelo_lab_8_reentrenado.h5')

test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)
predictions = model.predict(test_generator, steps=test_steps_per_epoch)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

accuracy = accuracy_score(true_classes, predicted_classes)
print(f'Accuracy: {accuracy * 100:.2f}%')

report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print('Classification Report:')
print(report)

<ipython-input-21-12f323ed4377>:6: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)


Accuracy: 69.65%
Classification Report:
                precision    recall  f1-score   support

     Adialer.C       0.69      1.00      0.81       122
     Agent.FYI       0.90      1.00      0.95       116
     Allaple.A       0.82      0.50      0.62      2949
     Allaple.L       0.45      0.95      0.61      1591
 Alueron.gen!J       0.97      0.94      0.96       198
     Autorun.K       0.00      0.00      0.00       106
       C2LOP.P       0.40      0.66      0.50       146
   C2LOP.gen!g       0.79      0.24      0.37       200
Dialplatform.B       0.96      0.99      0.97       177
     Dontovo.A       0.99      1.00      1.00       162
      Fakerean       0.97      0.99      0.98       381
 Instantaccess       0.85      1.00      0.92       431
    Lolyda.AA1       0.81      0.92      0.87       213
    Lolyda.AA2       0.97      0.76      0.85       184
    Lolyda.AA3       0.97      0.24      0.39       123
     Lolyda.AT       0.94      0.31      0.46       159
   Male

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Ataque y defensa de Deepfool

In [23]:
import numpy as np
from tensorflow.keras.models import load_model
from art.attacks.evasion import DeepFool
from art.estimators.classification import KerasClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tf.compat.v1.disable_eager_execution()

model = load_model('modelo_lab_8.h5')

classifier = KerasClassifier(model=model, clip_values=(0, 1))

deepfool_attack = DeepFool(classifier)

def generate_and_mix_adversarial_batch(generator, attack, batch_size=32):
    x_batch, y_batch = next(generator)
    x_adv = attack.generate(x=x_batch)

    x_train_mixed = np.concatenate((x_batch, x_adv), axis=0)
    y_train_mixed = np.concatenate((y_batch, y_batch), axis=0)

    indices = np.arange(x_train_mixed.shape[0])
    np.random.shuffle(indices)
    x_train_mixed = x_train_mixed[indices]
    y_train_mixed = y_train_mixed[indices]

    return x_train_mixed, y_train_mixed

x_train_mixed, y_train_mixed = generate_and_mix_adversarial_batch(train_generator, attack)

model.fit(x_train_mixed, y_train_mixed, epochs=10, batch_size=32)

model.save('modelo_lab_8_reentrenado.h5')

Train on 64 samples
Epoch 1/10
64/64 [==============================] - 0s 4ms/sample - loss: 1.6490 - accuracy: 0.7188
Epoch 2/10
64/64 [==============================] - 0s 478us/sample - loss: 0.8150 - accuracy: 0.7500
Epoch 3/10
64/64 [==============================] - 0s 453us/sample - loss: 0.5521 - accuracy: 0.8594
Epoch 4/10
64/64 [==============================] - 0s 460us/sample - loss: 0.3560 - accuracy: 0.8906
Epoch 5/10
64/64 [==============================] - 0s 474us/sample - loss: 0.3032 - accuracy: 0.8906
Epoch 6/10
64/64 [==============================] - 0s 368us/sample - loss: 0.2784 - accuracy: 0.9062
Epoch 7/10
64/64 [==============================] - 0s 372us/sample - loss: 0.2071 - accuracy: 0.9688
Epoch 8/10
64/64 [==============================] - 0s 376us/sample - loss: 0.1564 - accuracy: 0.9531
Epoch 9/10
64/64 [==============================] - 0s 333us/sample - loss: 0.2624 - accuracy: 0.9062
Epoch 10/10
64/64 [==============================] - 0s 532us/sa

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.models import load_model

model = load_model('modelo_lab_8_reentrenado.h5')

test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)
predictions = model.predict(test_generator, steps=test_steps_per_epoch)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

accuracy = accuracy_score(true_classes, predicted_classes)
print(f'Accuracy: {accuracy * 100:.2f}%')

report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print('Classification Report:')
print(report)

<ipython-input-24-12f323ed4377>:6: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)
/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Accuracy: 92.93%
Classification Report:
                precision    recall  f1-score   support

     Adialer.C       0.99      1.00      1.00       122
     Agent.FYI       1.00      0.97      0.98       116
     Allaple.A       0.94      1.00      0.97      2949
     Allaple.L       0.99      0.99      0.99      1591
 Alueron.gen!J       0.93      0.99      0.96       198
     Autorun.K       0.00      0.00      0.00       106
       C2LOP.P       0.70      0.71      0.70       146
   C2LOP.gen!g       0.91      0.49      0.64       200
Dialplatform.B       1.00      0.99      0.99       177
     Dontovo.A       0.89      1.00      0.94       162
      Fakerean       0.82      0.99      0.90       381
 Instantaccess       0.96      1.00      0.98       431
    Lolyda.AA1       0.99      0.85      0.91       213
    Lolyda.AA2       0.99      0.95      0.97       184
    Lolyda.AA3       1.00      0.99      1.00       123
     Lolyda.AT       1.00      0.96      0.98       159
   Male

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
